In [1]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm

with open('naver_total.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
    extracted_id = [item.get('id') for item in data if 'id' in item]

chrome_options = Options()
chrome_options.add_argument('--no-sandbox')

results = {}

for idx in tqdm(range(0, len(extracted_id), 100), desc='Processing batches'):
    batch_ids = extracted_id[idx : min(idx + 100, len(extracted_id))]

    with webdriver.Chrome(options=chrome_options) as driver:
        for id in tqdm(batch_ids, desc=f'Downloading info {idx}/{len(extracted_id)}', leave=False):
            hospital_info_elements = []

            try:
                driver.get(f'https://m.place.naver.com/place/{id}/home')
                driver.implicitly_wait(2)

                elements_to_click = [
                    '#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.pSavy > div > a > div > div > span',
                    '#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div.place_section_content > div > div:nth-child(8) > div > a > span.rvCSr'
                ]
                for element_selector in elements_to_click:
                    driver.find_element(By.CSS_SELECTOR, element_selector).click()
            except:
                pass

            elements_to_append = [
                '#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div > div > div.O8qbU.pSavy > div > a *',
                '#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div > div > div.O8qbU.WTjuc > div > ul a',
                '#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div > div > div:nth-child(8) > div > a > span.zPfVt'
            ]
            for element_selector in elements_to_append:
                try:
                    hospital_info_elements.extend([element.text for element in driver.find_elements(By.CSS_SELECTOR, element_selector)])
                except:
                    pass

            if hospital_info_elements:
                results[id] = hospital_info_elements
                
        with open(f'hospital_info.json', 'a+', encoding='utf-8') as file:
            json.dump(results, file, indent=4)
        results = {}

if results:
    with open('hospital_info.json', 'a+', encoding='utf-8') as file:
        json.dump(results, file, indent=4)

Processing batches:   0%|          | 0/48 [00:00<?, ?it/s]